In [ ]:
# This is the section to download Twitter data. The general code for retrieving this data is from the source
# https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a
# by Andrew Edward and does not reflect my own work.
# To utilise this code - you need to input your own bearer token and os.eniron token that you will obtain after gaining
# Approval from Twitter's API service
# In addition the keyword has to be updated accordingly or use the same bounded box for the Oxford Street District as used
# in the code. In addtion, the dates can be adjusted if you wish to use a different time period.

In [11]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# Other packages needed
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time

In [12]:
# Insert bearer token here
bearer_token = "AAAAAAAAAAAAAAAAAAAAAFHJQgEAAAAASJIemf%2BvOFvCBaLnhvInln%2FI6cM%3DJud6vLn4s22C2He8pbO97yXGcSPb7pjmS4DR22RvQjdfktooai"

In [13]:
# Insert bearer token here
os.environ['TOKEN'] ='AAAAAAAAAAAAAAAAAAAAAFHJQgEAAAAASJIemf%2BvOFvCBaLnhvInln%2FI6cM%3DJud6vLn4s22C2He8pbO97yXGcSPb7pjmS4DR22RvQjdfktooai'

In [14]:
def auth():
    return os.getenv('TOKEN')

In [15]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [16]:
def create_url(keyword, start_date, end_date, max_results = 10):
    search_url = "https://api.twitter.com/2/tweets/search/all"
    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [17]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [18]:
# Create file that the data will be read into
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author_id', 'created_at', 'tweet_id', 'lang', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'source', 'coordinates', 'text'])
csvFile.close()

In [19]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Tweet ID
        tweet_id = tweet['id']

        # 4. Language
        lang = tweet['lang']

        # 5. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 6. source
        source = tweet['source']
        
        # 7. coordinates
        if ('geo' in tweet):   
            coordinates = tweet['geo']
        else:
            coordinates = " "
        
        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, tweet_id, lang, retweet_count, reply_count, like_count, quote_count, source, coordinates, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [ ]:
#Inputs for tweets - keyword and dates can be adjusted
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "bounding_box: [-0.16 51.512 -0.13 51.52]"
start_list =    ['2019-01-01T00:00:00.000Z',
                 '2019-02-01T00:00:00.000Z',
                 '2019-03-01T00:00:00.000Z',
                 '2019-04-01T00:00:00.000Z',
                 '2019-05-01T00:00:00.000Z',
                 '2019-06-01T00:00:00.000Z',
                 '2019-07-01T00:00:00.000Z',
                 '2019-08-01T00:00:00.000Z',
                 '2019-09-01T00:00:00.000Z',
                 '2019-10-01T00:00:00.000Z',
                 '2019-11-01T00:00:00.000Z',
                 '2019-12-01T00:00:00.000Z',
                 '2020-01-01T00:00:00.000Z',
                 '2020-02-01T00:00:00.000Z',
                 '2020-03-01T00:00:00.000Z',
                 '2020-04-01T00:00:00.000Z',
                 '2020-05-01T00:00:00.000Z',
                 '2020-06-01T00:00:00.000Z',
                 '2020-07-01T00:00:00.000Z',
                 '2020-08-01T00:00:00.000Z',
                 '2020-09-01T00:00:00.000Z',
                 '2020-10-01T00:00:00.000Z',
                 '2020-11-01T00:00:00.000Z',
                 '2020-12-01T00:00:00.000Z',
                 '2021-01-01T00:00:00.000Z',
                 '2021-02-01T00:00:00.000Z',
                 '2021-03-01T00:00:00.000Z',
                 '2021-04-01T00:00:00.000Z',
                 '2021-05-01T00:00:00.000Z',
                 '2021-06-01T00:00:00.000Z',
                 '2021-07-01T00:00:00.000Z']

end_list =      ['2019-01-31T00:00:00.000Z',
                 '2019-02-28T00:00:00.000Z',
                 '2019-03-31T00:00:00.000Z',
                 '2019-04-30T00:00:00.000Z',
                 '2019-05-31T00:00:00.000Z',
                 '2019-06-30T00:00:00.000Z',
                 '2019-07-31T00:00:00.000Z',
                 '2019-08-31T00:00:00.000Z',
                 '2019-09-30T00:00:00.000Z',
                 '2019-10-31T00:00:00.000Z',
                 '2019-11-30T00:00:00.000Z',
                 '2019-12-31T00:00:00.000Z',
                 '2020-01-31T00:00:00.000Z',
                 '2020-02-29T00:00:00.000Z',
                 '2020-03-31T00:00:00.000Z',
                 '2020-04-30T00:00:00.000Z',
                 '2020-05-31T00:00:00.000Z',
                 '2020-06-30T00:00:00.000Z',
                 '2020-07-31T00:00:00.000Z',
                 '2020-08-31T00:00:00.000Z',
                 '2020-09-30T00:00:00.000Z',
                 '2020-10-31T00:00:00.000Z',
                 '2020-11-30T00:00:00.000Z',
                 '2020-12-31T00:00:00.000Z',
                 '2021-01-31T00:00:00.000Z',
                 '2021-02-28T00:00:00.000Z',
                 '2021-03-31T00:00:00.000Z',
                 '2021-04-30T00:00:00.000Z',
                 '2021-05-31T00:00:00.000Z',
                 '2021-06-30T00:00:00.000Z',
                 '2021-07-31T00:00:00.000Z']

max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'id','lang', 'retweet_count', 'reply_count', 'like_count','quote_count','source','coordinates','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 10000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)